<a href="https://colab.research.google.com/github/avyukthinna/ML_Lab/blob/main/1BM22CS060_BoostingEnsemble.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# BOOSTING ENSEMBLE

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

# Load your dataset from the local path
df = pd.read_csv("/content/data.csv")

# Drop any unnamed columns (like 'Unnamed: 32')
df.drop(columns=[col for col in df.columns if "Unnamed" in col], inplace=True)

# Encode labels: M = 1 (Malignant), B = -1 (Benign)
df['diagnosis'] = df['diagnosis'].map({'M' : 1, 'B': -1})

# Drop ID column if present
if 'id' in df.columns:
    df.drop(columns=['id'], inplace=True)

# Features and labels
X = df.drop(['diagnosis'], axis=1).values
y = df['diagnosis'].values

# Normalize features
X = (X - X.mean(axis=0)) / X.std(axis=0)

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Decision Stump class
class DecisionStump:
    def __init__(self):
        self.feature_index = None
        self.threshold = None
        self.polarity = 1
        self.alpha = None

    def predict(self, X):
        n = X.shape[0]
        predictions = np.ones(n)
        feature_values = X[:, self.feature_index]
        if self.polarity == 1:
            predictions[feature_values < self.threshold] = -1
        else:
            predictions[feature_values > self.threshold] = -1
        return predictions

# AdaBoost class
class AdaBoost:
    def __init__(self, n_clf=10):
        self.n_clf = n_clf
        self.clfs = []

    def fit(self, X, y):
        n_samples, n_features = X.shape
        weights = np.ones(n_samples) / n_samples

        for _ in range(self.n_clf):
            clf = DecisionStump()
            min_error = float('inf')

            for feature_i in range(n_features):
                feature_values = X[:, feature_i]
                thresholds = np.unique(feature_values)
                for threshold in thresholds:
                    for polarity in [1, -1]:
                        predictions = np.ones(n_samples)
                        if polarity == 1:
                            predictions[feature_values < threshold] = -1
                        else:
                            predictions[feature_values > threshold] = -1

                        error = np.sum(weights[y != predictions])

                        if error < min_error:
                            clf.polarity = polarity
                            clf.threshold = threshold
                            clf.feature_index = feature_i
                            min_error = error

            EPS = 1e-10
            clf.alpha = 0.5 * np.log((1 - min_error) / (min_error + EPS))
            predictions = clf.predict(X)
            weights *= np.exp(-clf.alpha * y * predictions)
            weights /= np.sum(weights)
            self.clfs.append(clf)

    def predict(self, X):
        clf_preds = [clf.alpha * clf.predict(X) for clf in self.clfs]
        y_pred = np.sum(clf_preds, axis=0)
        return np.sign(y_pred)

# Train and evaluate
model = AdaBoost(n_clf=20)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
accuracy = np.mean(y_pred == y_test)
print(f"\nTest Accuracy with 20 classifiers: {accuracy:.4f}")

# Print predictions and actual labels
print("\nPredicted labels for test set:")
print(y_pred)

print("\nActual labels for test set:")
print(y_test)

# Plot accuracy vs. number of classifiers
accuracies = []
for n in range(1, 31):
    model = AdaBoost(n_clf=n)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    acc = np.mean(y_pred == y_test)
    accuracies.append(acc)

plt.figure(figsize=(10, 6))
plt.plot(range(1, 31), accuracies, marker='o')
plt.title('AdaBoost Accuracy vs Number of Weak Classifiers')
plt.xlabel('Number of Weak Classifiers')
plt.ylabel('Test Accuracy')
plt.grid(True)
plt.tight_layout()
plt.show()